# 130 percent #

In [28]:
import pickle 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [46]:
def total_lawyers(week_assignments):
    """
    Recibe una lista de listas de asignaciones.
    Calcula cuantos abogados distintos fueron asignados.
    """
    lawyers = set()
    for assignment in week_assignments:
        lawyers.update(assignments)
    return len(lawyers)

In [30]:
with open('results/130percent/assignments.pickle', 'rb') as file:
    assignments = pickle.load(file)
    
with open('results/130percent/spr.pickle', 'rb') as file:
    spr = pickle.load(file)

with open('results/130percent/sr.pickle', 'rb') as file:
    sr = pickle.load(file)
    
with open('results/130percent/time_lawyers.pickle', 'rb') as file:
    lawyers_time = pickle.load(file)
    
mode1, mode2 = 'saa', 'greedy'

### $\lambda$ = 1.25, rate = 0.8 ###

In [34]:
lamb = 1.25
rate = 0.8

saa_assignment = assignments[lamb][rate][mode1]
greedy_assignment = assignments[lamb][rate][mode2]

saa_sr = sr[lamb][rate][mode1]
greedy_sr = sr[lamb][rate][mode2]

saa_spr = spr[lamb][rate][mode1]
greedy_spr = spr[lamb][rate][mode2]

saa_times = lawyers_time[lamb][rate][mode1]
greedy_times = lawyers_time[lamb][rate][mode2]

weeks = list(assignments[lamb][rate][mode1].keys())
services_per_week = {week: len(saa_assignment[week]) for week in weeks}
assignments_per_week = {week: total_lawyers()}

{1: 3, 2: 3, 4: 3, 5: 6, 8: 2, 9: 2, 10: 1, 11: 5, 12: 4, 15: 3}

In [35]:
greedy_times

{1: {(3, 1): 4.5,
  (3, 2): 6.5,
  (4, 1): 6.0,
  (4, 2): 6.0,
  (14, 1): 10.0,
  (14, 2): 10.0,
  (15, 1): 13.0,
  (15, 2): 13.0,
  (16, 1): 6.5,
  (16, 2): 6.5,
  (17, 1): 26.0,
  (17, 2): 26.0},
 2: {(0, 1): 0.0,
  (0, 2): 0.0,
  (0, 3): 0.0,
  (0, 4): 0.0,
  (0, 5): 0.0,
  (1, 1): 0.0,
  (1, 2): 0.0,
  (1, 3): 7.0,
  (1, 4): 7.0,
  (1, 5): 7.0,
  (2, 1): 0.9000000000000004,
  (2, 2): 0.9000000000000004,
  (2, 3): 3.9000000000000004,
  (2, 4): 3.9000000000000004,
  (2, 5): 3.9000000000000004},
 4: {(3, 1): 6.5,
  (3, 2): 6.5,
  (3, 3): 6.5,
  (4, 1): 4.0,
  (4, 2): 4.0,
  (4, 3): 13.0,
  (5, 1): 17.0,
  (5, 2): 17.0,
  (5, 3): 26.0,
  (6, 1): 6.5,
  (6, 2): 6.5,
  (6, 3): 6.5,
  (9, 1): 32.5,
  (9, 2): 32.5,
  (9, 3): 32.5,
  (12, 1): 7.0,
  (12, 2): 7.0,
  (12, 3): 7.0,
  (13, 1): 13.0,
  (13, 2): 13.0,
  (13, 3): 13.0},
 5: {(0, 1): 3.0,
  (0, 2): 3.0,
  (0, 3): 3.0,
  (0, 4): 3.0,
  (0, 5): 3.0,
  (0, 6): 3.0,
  (0, 7): 3.0,
  (0, 8): 3.0,
  (1, 1): 3.0,
  (1, 2): 3.0,
  (1, 3): 